# Visualization & Metrics

## Prerequisites

`Plotly` is used for all charts to provide an interactive experience: hover, toggle, and zoom. Reference the [Installation](installation.html#Plotting) notebook for information about configuring Plotly. However, static images are used in this notebook due to recent difficulty with 3rd party JS on the documentation portal.

## Overview

As described in the [Low-Level Docs](api_low_level.html#10.-Assess-the-Results.), the `Result(s)` of each training `Job` are automatically populated with metrics for each split/ fold of samples evaluated against the model. The `Algorithm.analysis_type` determines which metrics and plots are prepared:

* Although `'classification_multi'` and `'classification_binary'` share the same metrics and plots, they go about producing these artifacts differently. For example, ROC curves `roc_multi_class=None` vs `roc_multi_class='ovr'`.

* `'regression'`, unlike the classification analyses, does not have an 'accuracy' metric, so we substitute 'r2', R^2 (coefficient of determination, for it. There are no regression-specific plots.

---

We'll use the `datum` and `tests` modules to rapidly create some examples.

In [1]:
import aiqc
from aiqc import datum
from aiqc import tests

In [2]:
aiqc.destroy_db(True)
aiqc.setup()


=> Success - deleted database file at path:
/Users/BernardoFernandes/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - the following file path already exists on your system:
/Users/BernardoFernandes/Library/Application Support/aiqc/


=> Info - skipping folder creation as folder already exists at file path:
/Users/BernardoFernandes/Library/Application Support/aiqc/


=> Success - the following file path already exists on your system:
/Users/BernardoFernandes/Library/Application Support/aiqc/


=> Info - skipping as config file already exists at path:
/Users/BernardoFernandes/Library/Application Support/aiqc/config.json


=> Next run `aiqc.create_db()`.


=> Success - created database file at path:
/Users/BernardoFernandes/Library/Application Support/aiqc/aiqc.sqlite3


💾  Success - created all database tables.  💾



---

## Classification

In [3]:
queue_multiclass = tests.make_test_queue('keras_multiclass')


___/ featurecoder_index: 0 \_________

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.

['petal_width']

=> The remaining column(s) and dtype(s) can be used in downstream Featurecoder(s):
{'petal_length': 'float64', 'sepal_length': 'float64', 'sepal_width': 'float64'}


___/ featurecoder_index: 1 \_________

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.

['sepal_length', 'sepal_width', 'petal_length']

=> Done. All feature column(s) have encoder(s) associated with them.
No more Featurecoders can be added to this Encoderset.



In [ ]:
queue_multiclass.run_jobs()

🔮 Training Models 🔮:  88%|████████████████████████████████████▊     | 7/8 [00:18<00:02,  2.52s/it]

### Individual Job Metrics

Each training `Result` contains the following metrics.

In [ ]:
from pprint import pprint as p
p(
    queue_multiclass.jobs[0].results[0].metrics
)

It also contains per-epoch `History` metrics calculated during model training.

In [ ]:
queue_multiclass.jobs[0].results[0].history.keys()

### Aggregate Queue Metrics

* `selected_metrics:list` - If you get overwhelmed by the variety of metrics returned, then you can include the ones you want selectively by name.
* `sort_by:str` - You can sort the dataframe by any column name.
* `ascending:bool=False` - Descending if False.

In [ ]:
queue_multiclass.metrics_to_pandas(
    selected_metrics=None
    , sort_by=None
	, ascending=False
)

### Aggregate Queue Visualization

`plot_performance` aka the "boomerang chart" is unique to AIQC, and it really brings the benefits of the library to light. Each model from the Queue is evaluated against all splits/ folds.

When performing classification, the secondary training metric (non-loss) is 'accuracy'.

In [ ]:
queue_multiclass.plot_performance(
    max_loss = 1.0, min_accuracy = 0.80
)

![Classify Boomerang](docs/images/plot_classify_boomerang.png)

### Individual Job Visualization

Loss values in the first few epochs can often be extremely high before they plummet and become more gradual. This really stretches out the graph and makes it hard to see if the evaluation set is diverging or not. The `loss_skip_15pct:bool` parameter skips displaying the first 15% of epochs so that figure is more useful.

In [ ]:
queue_multiclass.jobs[0].results[0].plot_learning_curve(loss_skip_15pct=True)

![Classify Learn](docs/images/plot_classify_learn.png)

These classification metrics are preformatted for plotting.

In [ ]:
queue_multiclass.jobs[0].results[0].plot_data['test'].keys()

In [ ]:
queue_multiclass.jobs[0].results[0].plot_roc_curve()

![Classify ROC](docs/images/plot_roc.png)

In [ ]:
queue_multiclass.jobs[0].results[0].plot_confusion_matrix()

![Plot Confusion](docs/images/plot_confusion.png)

In [ ]:
queue_multiclass.jobs[0].results[0].plot_precision_recall()

![Precision Recall](docs/images/plot_precision_recall.png)

---

## Regression

In [ ]:
queue_regression = tests.make_test_queue('keras_regression')

In [ ]:
queue_regression.run_jobs()

### Individual Job Metrics

Each training `Result` contains the following metrics.

In [ ]:
from pprint import pprint as p
p(
    queue_regression.jobs[0].results[0].metrics
)

It also contains per-epoch metrics calculated during model training.

In [ ]:
queue_regression.jobs[0].results[0].history.keys()

### Aggregate Queue Metrics

In [ ]:
queue_regression.metrics_to_pandas()

### Aggregate Queue Visualization

When performing regression, the secondary training metric (non-loss) is 'r2'.

In [ ]:
queue_regression.plot_performance(
    max_loss=1.0, min_r2=0.60
)

![Regression Boomerang](docs/images/plot_regression_boomerang.png)

### Individual Job Visualization

In [ ]:
queue_regression.jobs[0].results[0].plot_learning_curve(loss_skip_15pct=True)

![Regression Learn](docs/images/plot_regression_learn.png)